In [2]:
# !pip install --upgrade nba_api

In [17]:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns; sns.set(style='whitegrid')
import numpy as np
import pandas as pd
import pickle
import tqdm
import glob
import os
import sys

from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import playerprofilev2, teaminfocommon, commonteamroster, playercareerstats, commonplayerinfo, commonallplayers
from nba_api.stats.library.parameters import Season, SeasonType, LeagueIDNullable
from nba_api.stats.library.eventmsgtype import EventMsgType

In [2]:
df_teams = pd.DataFrame(teams.get_teams())
print(df_teams.shape)
df_teams.head()

(30, 7)


,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


In [23]:
def get_current_rosters(team_ids):
    return pd.concat([(commonteamroster
                       .CommonTeamRoster(team_id=team_id)
                       .get_data_frames()[0]
                      )
                      for team_id in tqdm.tqdm(team_ids)])

rosters = get_current_rosters(df_teams.id)
print(rosters.shape)
rosters.head()

100%|██████████| 30/30 [00:18<00:00,  1.62it/s]

(493, 14)


,TeamID,SEASON,LeagueID,PLAYER,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID
0,1610612737,2020,00,Brandon Goodwin,brandon-goodwin,0,G,6-0,180,"OCT 02, 1995",25.0,2,Florida Gulf Coast,1629164
1,1610612737,2020,00,Nathan Knight,nathan-knight,1,F-C,6-10,253,"SEP 20, 1997",23.0,R,William & Mary,1630233
2,1610612737,2020,00,Kevin Huerter,kevin-huerter,3,G-F,6-7,190,"AUG 27, 1998",22.0,2,Maryland,1628989
3,1610612737,2020,00,Skylar Mays,skylar-mays,4,G,6-4,205,"SEP 05, 1997",23.0,R,Louisiana State,1630219
4,1610612737,2020,00,Rajon Rondo,rajon-rondo,7,G,6-1,180,"FEB 22, 1986",35.0,14,Kentucky,200765


In [28]:
rosters.to_pickle('rosters_20210317.pickle.zip', compression='zip')

In [30]:
def get_current_team_stats(team_ids, season_nullable='2020-21'):
    return pd.concat([(teaminfocommon
                       .TeamInfoCommon(team_id=team_id,
                                       season_nullable=season_nullable)
                       .get_data_frames()[1]
                      )
                      for team_id in tqdm.tqdm(team_ids)])

tstats = get_current_team_stats(df_teams.id)
print(tstats.shape)
tstats.head()



  0%|          | 0/30 [00:00<?, ?it/s]

  3%|▎         | 1/30 [00:00<00:19,  1.46it/s]

  7%|▋         | 2/30 [00:01<00:16,  1.67it/s]

 10%|█         | 3/30 [00:01<00:15,  1.79it/s]

 13%|█▎        | 4/30 [00:01<00:12,  2.01it/s]

 17%|█▋        | 5/30 [00:02<00:11,  2.10it/s]

 20%|██        | 6/30 [00:02<00:12,  1.98it/s]

 23%|██▎       | 7/30 [00:03<00:10,  2.17it/s]

 27%|██▋       | 8/30 [00:03<00:09,  2.29it/s]

 30%|███       | 9/30 [00:04<00:09,  2.20it/s]

 33%|███▎      | 10/30 [00:04<00:08,  2.27it/s]

 37%|███▋      | 11/30 [00:04<00:07,  2.41it/s]

 40%|████      | 12/30 [00:05<00:07,  2.41it/s]

 43%|████▎     | 13/30 [00:06<00:09,  1.81it/s]

 47%|████▋     | 14/30 [00:08<00:17,  1.11s/it]

 50%|█████     | 15/30 [00:09<00:16,  1.08s/it]

 53%|█████▎    | 16/30 [00:10<00:15,  1.09s/it]

 57%|█████▋    | 17/30 [00:12<00:17,  1.38s/it]

 60%|██████    | 18/30 [00:13<00:14,  1.19s/it]

 63%|██████▎   | 19/30 [00:13<00:10,  1.06it/s]

 67%|██████▋   | 20/30 [00:14<00:08,

(30, 11)


,LEAGUE_ID,SEASON_ID,TEAM_ID,PTS_RANK,PTS_PG,REB_RANK,REB_PG,AST_RANK,AST_PG,OPP_PTS_RANK,OPP_PTS_PG
0,00,22020,1610612737,14,112.9,7,45.8,20,24.3,12,111.4
0,00,22020,1610612738,19,112.1,15,44.0,26,22.7,10,111.1
0,00,22020,1610612739,30,103.3,21,43.2,23,23.2,16,111.9
0,00,22020,1610612740,4,115.9,4,46.6,9,26.3,27,115.7
0,00,22020,1610612741,11,114.4,10,44.6,8,26.3,24,114.3


In [31]:
tstats.to_pickle('team_stats_20210317.pickle.zip', compression='zip')

In [21]:
tt.get_data_frames()[1]#.head()

,LEAGUE_ID,SEASON_ID,TEAM_ID,PTS_RANK,PTS_PG,REB_RANK,REB_PG,AST_RANK,AST_PG,OPP_PTS_RANK,OPP_PTS_PG
0,00,22020,1610612737,14,112.9,7,45.8,20,24.3,12,111.4


In [32]:
all_dfs = [] # 2936/4501

In [38]:
len(all_dfs)

493

In [37]:
for player_id in tqdm.notebook.tqdm(rosters.PLAYER_ID[len(all_dfs):]):
    st = playercareerstats.PlayerCareerStats(player_id=player_id)
    all_dfs.append(st.season_totals_regular_season.get_data_frame())

/home/noone/.virtualenvs/env36/lib/python3.6/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [39]:
result_df = pd.concat(all_dfs, axis=0)
print(result_df.shape)
result_df.to_pickle('all_players_seasons_stats_20210317.pickle.zip', compression='zip')

(2893, 27)


In [40]:
result_df.sample(10)

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
4,1626164,2019-20,00,1610612756,PHX,23.0,70,70,2512.0,627,...,0.919,29,268,297,456,49,18,264,213,1863
3,203897,2017-18,00,1610612741,CHI,23.0,24,24,656.0,136,...,0.813,9,85,94,72,24,4,43,55,401
1,1626171,2016-17,00,1610612741,CHI,22.0,64,13,1001.0,183,...,0.661,75,221,296,35,16,11,37,94,437
1,1628404,2018-19,00,1610612747,LAL,24.0,67,22,1715.0,189,...,0.688,35,213,248,93,64,40,58,147,525
9,201145,2014-15,00,1610612763,MEM,28.0,45,37,1361.0,210,...,0.825,43,144,187,81,26,21,52,81,588
8,202696,2019-20,00,1610612753,ORL,29.0,62,62,1998.0,493,...,0.784,140,533,673,222,54,47,88,135,1215
11,202693,2018-19,00,0,TOT,29.0,58,16,1271.0,204,...,0.772,61,204,265,79,36,22,54,175,547
2,1626181,2017-18,00,1610612761,TOR,25.0,70,18,1062.0,150,...,0.821,14,105,119,89,37,16,66,111,385
6,1626153,2019-20,00,1610612742,DAL,28.0,73,5,1570.0,190,...,0.770,70,210,280,244,85,22,73,93,504
8,201988,2017-18,00,1610612759,SAS,29.0,82,36,2107.0,280,...,0.890,26,128,154,229,55,11,106,128,819


In [41]:
result_df.LEAGUE_ID.value_counts().sort_index()

00    2893
Name: LEAGUE_ID, dtype: int64

In [42]:
sum(d.empty for d in all_dfs)

3